# Implementation

## Setting up Tensorflow
Restart runtime, set it to GPU and then run this chunk. The default is tf2.x which has slightly different functions. I prefer working with tf1.x

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


## Loading Dataset
Loading the EEG dataset from physionet databases [1]. Physionet allows us to load dataset directly from cloud. 

In [ ]:
!gsutil -m cp -r gs://chbmit-1.0.0.physionet.org data

CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a di

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Preproccessing
Some basic data preprocessing includes filtering and obtaining signals in frequency domain usinf fft. 

## Building a CNN


Here we just import some libraries and use them to buid an architecture. 

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical

# x_train = trainX.reshape(trainX.shape)
# x_test = testX.reshape(testX.shape)

# x_train = (x_train - 255)/255
# x_test = (x_test - 255)/255

# y_train = to_categorical(trainy)
# y_test = to_categorical(testy)

in1 = Input(shape=x_train[0].shape)
c1 = Conv2D(256, (5,5), activation='relu')(in1)
m1 = MaxPooling2D()(c1)
# c2 = Conv2D(128, (5,5), activation='relu')(m1)
# # m2 = MaxPooling2D()(c2)
# d1 = Dropout(0.7)(c2)
# c3 = Conv2D(256, (5,5), activation='relu')(d1)
# # m3 = MaxPooling2D()(c3)
# d2 = Dropout(0.8)(c3)
# # c4 = Conv2D(64, (3,3), activation='relu')(d2)
# # m4 = MaxPooling2D()(c4)
# fl = Flatten()(d2)
# d1 = Dense(64, activation='relu')(fl)
# d2 = Dense(16, activation='relu')(d1)
o = Dense(len(y_train[0]), activation='softmax')(d2)

model = Model(inputs=in1, outputs=o)
print(model.summary())

Now after we have constructed our model let's train it. 

In [ ]:
model = Model(inputs = in1, outputs = o)
model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics=['acc'])
history_cnn = model.fit(x_train, y_train, epochs = 60, batch_size = 32, validation_data = (x_test, y_test))

## Results

In [ ]:
loss = history_cnn.history['loss']
val_loss = history_cnn.history['val_loss']
epochs = range(1, len(loss) + 1)
pyplot.grid()
pyplot.plot(epochs, loss, 'y', label='Training loss')
pyplot.plot(epochs, val_loss, 'r', label='Validation loss')
pyplot.title('Training and validation loss')
pyplot.xlabel('Epochs')
pyplot.ylabel('Loss')
pyplot.legend()
pyplot.show()

pyplot.grid()
acc = history_cnn.history['acc']
val_acc = history_cnn.history['val_acc']
epochs = range(1, len(loss) + 1)
pyplot.plot(epochs, acc, 'y', label='Training Accuracy')
pyplot.plot(epochs, val_acc, 'r', label='Validation Accuracy')
pyplot.title('Training and validation Accuracies')
pyplot.xlabel('Epochs')
pyplot.ylabel('Loss')
pyplot.legend()
pyplot.show()

## Discussion and Conclusion

Yay our implementation is successful...

# Bibliography
[1] CHB-MIT Scalp EEG Database, Retrieved from: https://physionet.org/content/chbmit/1.0.0/
